<a href="https://colab.research.google.com/github/Hidenori24/Signate_colab/blob/main/SMBC_GreenChallenge_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## importとシードなどの設定

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
# 日本語フォントを簡単に使う
!pip -q install japanize-matplotlib
import japanize_matplotlib

# シードの設定
np.random.seed(42)
tf.random.set_seed(42)

## Driveへの接続、データの取得

In [34]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/ML/Signate_1634/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### train/test/sample_submission の取り込み

In [35]:
train_df  = pd.read_csv(data_path + 'train.csv', parse_dates=['time'], index_col='time')
df = train_df
test_df   = pd.read_csv(data_path + 'test.csv', parse_dates=['time'], index_col='time')
sample_submission_df = pd.read_csv(data_path + 'sample_submit.csv', header=None)  # header 無し

### 概要確認

In [36]:
# 概要を確認
print("\n--- 学習データ (train_df) ---")
display(train_df.head())
print(f"shape: {train_df.shape}")

print("\n--- テストデータ (test_df) ---")
display(test_df.head())
print(f"shape: {test_df.shape}")

print("\n--- サンプル提出 (sample_submission_df) ---")
display(sample_submission_df.head())
print(f"shape: {sample_submission_df.shape}")


--- 学習データ (train_df) ---


,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run_of_river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,seville_rain_1h,seville_rain_3h,seville_snow_3h,seville_clouds_all,seville_weather_id,seville_weather_main,seville_weather_description,seville_weather_icon,price_actual,item_ID
time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 00:00:00+01:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,64.02,spain_total
2015-01-02 01:00:00+01:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,58.46,spain_total
2015-01-02 02:00:00+01:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,54.70,spain_total
2015-01-02 03:00:00+01:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,54.91,spain_total
2015-01-02 04:00:00+01:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,53.07,spain_total


shape: (26280, 92)

--- テストデータ (test_df) ---


,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run_of_river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,seville_wind_deg,seville_rain_1h,seville_rain_3h,seville_snow_3h,seville_clouds_all,seville_weather_id,seville_weather_main,seville_weather_description,seville_weather_icon,item_ID
time,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+01:00,279.0,0.0,3927.0,895.0,189.0,230.0,1069.0,1893.0,7104.0,53.0,...,343,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 01:00:00+01:00,282.0,0.0,3948.0,878.0,177.0,1269.0,1058.0,1024.0,7101.0,52.0,...,343,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 02:00:00+01:00,283.0,0.0,3791.0,890.0,175.0,2197.0,1052.0,888.0,7100.0,52.0,...,0,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 03:00:00+01:00,280.0,0.0,3671.0,881.0,175.0,2965.0,1032.0,645.0,7101.0,53.0,...,40,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 04:00:00+01:00,286.0,0.0,3460.0,861.0,173.0,2705.0,1001.0,661.0,7101.0,53.0,...,30,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total


shape: (8760, 91)

--- サンプル提出 (sample_submission_df) ---


,0,1
0,2018-01-01 00:00:00+01:00,0
1,2018-01-01 01:00:00+01:00,0
2,2018-01-01 02:00:00+01:00,0
3,2018-01-01 03:00:00+01:00,0
4,2018-01-01 04:00:00+01:00,0


shape: (8760, 2)


## データの前処理

---



In [37]:
# 基本情報を確認
print(f"データのサイズ: {df.shape}")
print(f"期間: {df.index.min()} から {df.index.max()}")
print(f"推論対象の期間: {test_df.index.min()} から {test_df.index.max()}")
print(f"⽋損値の合計: {df.isna().sum().sum()}")
# ⽋損値を処理
# 時系列データなので、前⽅補間が適切
#df = df.fillna(method='ffill')
df = df.ffill()
# まだ⽋損がある場合は後⽅補間
#df = df.fillna(method='bfill')
df = df.bfill()
# ⽬的変数の確認
print("\n⽬的変数 'price_actual' の統計:")
print(df['price_actual'].describe())
# 価格の時系列プロット
plt.figure(figsize=(15, 6))
plt.plot(df.index, df['price_actual'])
plt.title('電⼒価格の推移')
plt.ylabel('価格')
plt.grid(True)
plt.tight_layout()
plt.savefig('price_time_series.png')
plt.close()
# 説明変数を準備
# item_IDはスペインの合計データのみ使⽤
df = df[df['item_ID'] == 'spain_total']
# 不要な列を削除
X = df.drop(['price_actual', 'item_ID'], axis=1)
# ⽬的変数
y = df['price_actual']

データのサイズ: (26280, 92)
期間: 2015-01-02 00:00:00+01:00 から 2017-12-31 23:00:00+01:00
推論対象の期間: 2018-01-01 00:00:00+01:00 から 2018-12-31 23:00:00+01:00
⽋損値の合計: 321

⽬的変数 'price_actual' の統計:
count    26280.000000
mean        56.028338
std         14.340356
min          9.330000
25%         47.617500
50%         55.930000
75%         65.192500
max        116.800000
Name: price_actual, dtype: float64


## 特徴量エンジニアリング

In [38]:
def create_additional_features(df):
    """
    時間ベースの特徴量とラグ特徴量を作成
    """
    df_features = df.copy()

    # インデックスがDatetimeIndexであることを確認し、必要なら変換
    if not isinstance(df_features.index, pd.DatetimeIndex) or df_features.index.tz is not None:
        df_features.index = pd.to_datetime(df_features.index, utc=True)

    # ここから既存の時間ベースの特徴量作成コード
    df_features['hour'] = df_features.index.hour
    df_features['dayofweek'] = df_features.index.dayofweek
    df_features['month'] = df_features.index.month
    df_features['year'] = df_features.index.year
    df_features['quarter'] = df_features.index.quarter
    df_features['dayofyear'] = df_features.index.dayofyear
    # ISO週番号の取得方法が古いバージョンのPandasで異なる場合があります。
    # .weekofyear は非推奨になったため、.isocalendar().week を使用するのが推奨です。
    # ただし、もし古いPandasバージョンでisocalendarが使えない場合は .weekofyear を試してください。
    try:
        df_features['weekofyear'] = df_features.index.isocalendar().week.astype(int) # ISO週番号
    except AttributeError:
        # もし isocalendar がない場合は、古い .weekofyear を試す
        print("警告: .isocalendar() が見つかりません。代わりに .weekofyear を使用します。Pandasのバージョンを確認してください。")
        df_features['weekofyear'] = df_features.index.weekofyear.astype(int)

    df_features['dayofmonth'] = df_features.index.day
    df_features['is_weekend'] = (df_features.index.dayofweek >= 5).astype(int)

    # ラグ特徴量（例: 1時間前、24時間前、48時間前、1週間前）
    # ... (既存のラグ特徴量作成コード)
    df_features['price_actual_lag_1'] = df_features['price_actual'].shift(1)
    df_features['price_actual_lag_24'] = df_features['price_actual'].shift(24)
    df_features['price_actual_lag_48'] = df_features['price_actual'].shift(48)
    df_features['price_actual_lag_168'] = df_features['price_actual'].shift(168) # 1週間前

    # 必要に応じて他の特徴量のラグも追加できます。
    # 例: df_features['consumption_lag_24'] = df_features['consumption'].shift(24)

    # 移動平均（例: 24時間の移動平均）
    df_features['price_actual_rolling_mean_24'] = df_features['price_actual'].rolling(window=24).mean()

    # 指数平滑化（例: span=24の指数平滑化）
    #df_features['price_actual_ewm_24'] = df_features['price_actual'].ewm(span=24).mean()


    # 新たに生成された特徴量によって発生する可能性のある欠損値を処理
    df_features = df_features.ffill()
    df_features = df_features.bfill()

    return df_features

## 特徴量選択と次元削減

In [39]:
# 特徴量選択（f_regressionを使って上位の特徴を選択）
def select_features_and_scale(X, y, k=30):
    """
    特徴量選択前に標準化を行い、選択された特徴量を返す
    """
    # 数値型以外のカラムを削除
    X_numeric = X.select_dtypes(include=np.number)

    # 標準化
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_numeric)
    X_scaled_df = pd.DataFrame(X_scaled, index=X_numeric.index, columns=X_numeric.columns) # DataFrameに戻す

    # もし数値型カラムがkより少ない場合はkを調整
    k = min(k, X_scaled_df.shape[1])

    selector = SelectKBest(score_func=f_regression, k=k)
    X_selected_scaled = selector.fit_transform(X_scaled_df, y)

    # 選択された特徴のインデックスを取得
    selected_indices = selector.get_support(indices=True)
    selected_features = X_scaled_df.columns[selected_indices]
    print(f"標準化後、選択された特徴量 ({k}個): {selected_features.tolist()}")

    # 選択された特徴量を DataFrame として返す（スケール済み）
    return pd.DataFrame(X_selected_scaled, index=X_numeric.index, columns=selected_features), scaler, selector

# PCAで次元削減 (選択された特徴量に対して適用)
def apply_pca_on_selected(X_selected_scaled, n_components=15):
  # X_selected_scaled はすでに StandardScaler でスケール済みを想定
  pca = PCA(n_components=n_components)
  X_pca = pca.fit_transform(X_selected_scaled)

  # 説明された分散の比率を確認
  explained_variance = pca.explained_variance_ratio_.sum()
  print(f"PCAにより{n_components}成分で{explained_variance:.2%}の分散を説明 (選択特徴量から)")
  return X_pca, pca

# 特徴量エンジニアリングを適用
df_featured = create_additional_features(df.copy())

# 特徴量エンジニアリング後のデータから説明変数と目的変数を作成
X_featured = df_featured.drop(['price_actual', 'item_ID'], axis=1)
y_featured = df_featured['price_actual']

# 特徴量選択と標準化を適用
X_selected_scaled, feature_scaler, feature_selector = select_features_and_scale(X_featured, y_featured, k=30)

# 選択された特徴量にPCAを適用
X_pca, pca = apply_pca_on_selected(X_selected_scaled, n_components=15)

# ✅ ここで履歴長を定義
seq_length = 24

# 相関ヒートマップ（選択された特徴量 - スケール済み）
# X_selected_scaled が DataFrame なのでそのまま corr() を使用
plt.figure(figsize=(15, 12))
correlation = X_selected_scaled.corr()
mask = np.triu(np.ones_like(correlation, dtype=bool))
sns.heatmap(correlation, mask=mask, annot=False, cmap='coolwarm', linewidths=0.5)
plt.title('標準化・選択された特徴量の相関ヒートマップ')
plt.tight_layout()
plt.savefig('selected_feature_correlation.png')
plt.close()

# PCA を外す場合のデータ準備 (スケール済み・選択済みだが PCA なし)
X_no_pca = X_selected_scaled.values # numpy 配列として取得
print(f"PCA適用なしデータ形状: {X_no_pca.shape}")

標準化後、選択された特徴量 (30個): ['generation_biomass', 'generation_fossil_brown_coal/lignite', 'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil', 'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage', 'generation_other', 'total_load_actual', 'valencia_wind_speed', 'madrid_wind_speed', 'bilbao_pressure', 'bilbao_wind_speed', 'bilbao_wind_deg', 'bilbao_clouds_all', 'barcelona_wind_speed', 'seville_pressure', 'seville_wind_deg', 'hour', 'dayofweek', 'month', 'quarter', 'dayofyear', 'weekofyear', 'is_weekend', 'price_actual_lag_1', 'price_actual_lag_24', 'price_actual_lag_48', 'price_actual_lag_168', 'price_actual_rolling_mean_24']
PCAにより15成分で88.59%の分散を説明 (選択特徴量から)
PCA適用なしデータ形状: (26280, 30)


## 時系列データの準備

In [40]:
def create_sequences(X, y, seq_length=24):
    """
    時系列データのシーケンスを作成
    seq_length: 何時間前までのデータを使うか（例：24時間）
    """
    X_seq, y_seq = [], []

    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])

    return np.array(X_seq), np.array(y_seq)

# スケーリング
y_scaler = MinMaxScaler()
y_scaled = y_scaler.fit_transform(y_featured.values.reshape(-1, 1))

# データセットの分割割合
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

train_size = int(len(X_featured) * train_ratio)
val_size = int(len(X_featured) * val_ratio)
test_size = len(X_featured) - train_size - val_size # 残りをテストサイズとする

# -------------------------------------------------------------
# PCA適用データでの分割とシーケンス作成
# -------------------------------------------------------------
print("--- PCA適用データ ---")
X_train_pca = X_pca[:train_size]
X_val_pca = X_pca[train_size:train_size+val_size]
X_test_pca = X_pca[train_size+val_size:train_size+val_size+test_size]

y_train_scaled_pca = y_scaled[:train_size]
y_val_scaled_pca = y_scaled[train_size:train_size+val_size]
y_test_scaled_pca = y_scaled[train_size+val_size:train_size+val_size+test_size]


X_train_seq_pca, y_train_seq_pca = create_sequences(X_train_pca, y_train_scaled_pca, seq_length)
X_val_seq_pca, y_val_seq_pca = create_sequences(X_val_pca, y_val_scaled_pca, seq_length)
X_test_seq_pca, y_test_seq_pca = create_sequences(X_test_pca, y_test_scaled_pca, seq_length)

print(f"PCA適用トレーニングデータ形状: {X_train_seq_pca.shape}, {y_train_seq_pca.shape}")
print(f"PCA適用検証データ形状: {X_val_seq_pca.shape}, {y_val_seq_pca.shape}")
print(f"PCA適用テストデータ形状: {X_test_seq_pca.shape}, {y_test_seq_pca.shape}")


# -------------------------------------------------------------
# PCA適用なしデータでの分割とシーケンス作成
# X_no_pca はすでに select_features_and_scale でスケール済み
# -------------------------------------------------------------
print("\n--- PCA適用なしデータ ---")
X_train_no_pca = X_no_pca[:train_size]
X_val_no_pca = X_no_pca[train_size:train_size+val_size]
X_test_no_pca = X_no_pca[train_size+val_size:train_size+val_size+test_size]

# 目的変数は共通のスケーリング済みデータを使用
y_train_scaled_no_pca = y_scaled[:train_size]
y_val_scaled_no_pca = y_scaled[train_size:train_size+val_size]
y_test_scaled_no_pca = y_scaled[train_size+val_size:train_size+val_size+test_size]


X_train_seq_no_pca, y_train_seq_no_pca = create_sequences(X_train_no_pca, y_train_scaled_no_pca, seq_length)
X_val_seq_no_pca, y_val_seq_no_pca = create_sequences(X_val_no_pca, y_val_scaled_no_pca, seq_length)
X_test_seq_no_pca, y_test_seq_no_pca = create_sequences(X_test_no_pca, y_test_scaled_no_pca, seq_length)


print(f"PCAなしトレーニングデータ形状: {X_train_seq_no_pca.shape}, {y_train_seq_no_pca.shape}")
print(f"PCAなし検証データ形状: {X_val_seq_no_pca.shape}, {y_val_seq_no_pca.shape}")
print(f"PCAなしテストデータ形状: {X_test_seq_no_pca.shape}, {y_test_seq_no_pca.shape}")

--- PCA適用データ ---
PCA適用トレーニングデータ形状: (21000, 24, 15), (21000, 1)
PCA適用検証データ形状: (2604, 24, 15), (2604, 1)
PCA適用テストデータ形状: (2604, 24, 15), (2604, 1)

--- PCA適用なしデータ ---
PCAなしトレーニングデータ形状: (21000, 24, 30), (21000, 1)
PCAなし検証データ形状: (2604, 24, 30), (2604, 1)
PCAなしテストデータ形状: (2604, 24, 30), (2604, 1)


## モデル構築と学習

### LSTMモデル (PCA適用データ)

In [41]:
def build_lstm_model(seq_length, n_features):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(seq_length, n_features)),
        BatchNormalization(),
        Dropout(0.3),

        LSTM(128, return_sequences=False),
        BatchNormalization(),
        Dropout(0.3),

        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# LSTMモデルの構築 (PCA適用データ)
lstm_model_pca = build_lstm_model(seq_length, X_train_seq_pca.shape[2])
print("\n--- LSTMモデル (PCA適用) ---")
print(lstm_model_pca.summary())

# コールバックの設定
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    verbose=1,
    restore_best_weights=True
)

checkpoint_pca = ModelCheckpoint(
    data_path + 'lstm_model_pca.h5', # ファイル名を変更して区別
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.0001,
    verbose=1
)

# モデルの学習 (PCA適用データ)
print("\n--- LSTMモデル学習開始 (PCA適用) ---")
lstm_history_pca = lstm_model_pca.fit(
    X_train_seq_pca, y_train_seq_pca,
    epochs=100,
    batch_size=64,
    validation_data=(X_val_seq_pca, y_val_seq_pca),
    callbacks=[early_stopping, checkpoint_pca, reduce_lr],
    verbose=1
)

# 学習履歴のプロット部分 (PCA適用)
plt.figure(figsize=(7, 5))

plt.plot(lstm_history_pca.history['loss'], label='Training Loss')
plt.plot(lstm_history_pca.history['val_loss'], label='Validation Loss')
plt.title('LSTM (PCA) - Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('lstm_training_history_loss_pca.png') # ファイル名を変更
plt.close()



--- LSTMモデル (PCA適用) ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                  │ (None, 24, 64)         │        20,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 24, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 124,225 (485.25 KB)

...

329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0127 - val_loss: 0.0040 - learning_rate: 0.0010
Epoch 4/100
327/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0108
Epoch 4: val_loss improved from 0.00403 to 0.00367, saving model to /content/drive/MyDrive/ML/Signate_1634/lstm_model_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0108 - val_loss: 0.0037 - learning_rate: 0.0010
Epoch 5/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0097
Epoch 5: val_loss improved from 0.00367 to 0.00246, saving model to /content/drive/MyDrive/ML/Signate_1634/lstm_model_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0097 - val_loss: 0.0025 - learning_rate: 0.0010
Epoch 6/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0087
Epoch 6: val_loss improved from 0.00246 to 0.00197, saving model to /content/drive/MyDrive/ML/Signate_1634/lstm_model_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0087 - val_loss: 0.0020 - learning_rate: 0.0010
Epoch 7/100
326/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0082
Epoch 7: val_loss did not improve from 0.00197
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0082 - val_loss: 0.0023 - learning_rate: 0.0010
Epoch 8/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0084
Epoch 8: val_loss did not improve from 0.00197
329/329 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0084 - val_loss: 0.0031 - learning_rate: 0.0010
Epoch 9/100
326/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0074
Epoch 9: val_loss did not improve from 0.00197
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0074 - val_loss: 0.0031 - learning_rate: 0.0010
Epoch 10/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0074
Epoch 10: val_loss improved from 0.00197 to 0.00133, saving model to /content/drive/MyDrive/ML/Signate_1634/lstm_model_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0074 - val_loss: 0.0013 - learning_rate: 0.0010
Epoch 11/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0064
Epoch 11: val_loss did not improve from 0.00133
329/329 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0064 - val_loss: 0.0059 - learning_rate: 0.0010
Epoch 12/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0079
Epoch 12: val_loss did not improve from 0.00133
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0079 - val_loss: 0.0024 - learning_rate: 0.0010
Epoch 13/100
326/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0063
Epoch 13: val_loss did not improve from 0.00133
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0063 - val_loss: 0.0018 - learning_rate: 0.0010
Epoch 14/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0061
Epoch 14: val_loss did not improve from 0.00133
329/329 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.0061 - val_loss: 0.0021 - learning_rate: 0.0010
Epoch 15/100
324/329 ━━━━━━━━━━━

### LSTMモデル (PCA適用なし)

In [42]:
# LSTMモデルの構築 (PCA適用なしデータ)
# n_features が X_train_seq_no_pca の特徴量数になる
lstm_model_no_pca = build_lstm_model(seq_length, X_train_seq_no_pca.shape[2])
print("\n--- LSTMモデル (PCA適用なし) ---")
print(lstm_model_no_pca.summary())

checkpoint_no_pca = ModelCheckpoint(
    data_path + 'lstm_model_no_pca.h5', # ファイル名を変更して区別
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

# モデルの学習 (PCA適用なしデータ)
print("\n--- LSTMモデル学習開始 (PCA適用なし) ---")
lstm_history_no_pca = lstm_model_no_pca.fit(
    X_train_seq_no_pca, y_train_seq_no_pca,
    epochs=100,
    batch_size=64,
    validation_data=(X_val_seq_no_pca, y_val_seq_no_pca),
    callbacks=[early_stopping, checkpoint_no_pca, reduce_lr], # 同じコールバックを使用
    verbose=1
)

# 学習履歴のプロット部分 (PCA適用なし)
plt.figure(figsize=(7, 5))

plt.plot(lstm_history_no_pca.history['loss'], label='Training Loss')
plt.plot(lstm_history_no_pca.history['val_loss'], label='Validation Loss')
plt.title('LSTM (No PCA) - Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('lstm_training_history_loss_no_pca.png') # ファイル名を変更
plt.close()



--- LSTMモデル (PCA適用なし) ---


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_14 (LSTM)                  │ (None, 24, 64)         │        24,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 24, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_15 (LSTM)                  │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 128,065 (500.25 KB)

 Trainable params: 127,681 (498.75 KB)

...

329/329 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.0022 - val_loss: 8.6169e-04 - learning_rate: 1.0000e-04
Epoch 37/100
325/329 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0022
Epoch 37: val_loss did not improve from 0.00086
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0022 - val_loss: 8.8649e-04 - learning_rate: 1.0000e-04
Epoch 38/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0021
Epoch 38: val_loss did not improve from 0.00086
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0021 - val_loss: 9.1242e-04 - learning_rate: 1.0000e-04
Epoch 39/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0022
Epoch 39: val_loss did not improve from 0.00086
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0022 - val_loss: 8.8602e-04 - learning_rate: 1.0000e-04
Epoch 40/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0021
Epoch 40: val_loss did not improve from 0.00086
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0021 - val_loss: 8.8496e-04 - learning_rate: 1.000

329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0019 - val_loss: 8.4320e-04 - learning_rate: 1.0000e-04
Epoch 49/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0019
Epoch 49: val_loss did not improve from 0.00084
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0019 - val_loss: 8.6287e-04 - learning_rate: 1.0000e-04
Epoch 50/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0020
Epoch 50: val_loss improved from 0.00084 to 0.00082, saving model to /content/drive/MyDrive/ML/Signate_1634/lstm_model_no_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0020 - val_loss: 8.1875e-04 - learning_rate: 1.0000e-04
Epoch 51/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0019
Epoch 51: val_loss improved from 0.00082 to 0.00081, saving model to /content/drive/MyDrive/ML/Signate_1634/lstm_model_no_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0019 - val_loss: 8.0938e-04 - learning_rate: 1.0000e-04
Epoch 52/100
323/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0019
Epoch 52: val_loss did not improve from 0.00081
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0019 - val_loss: 8.4961e-04 - learning_rate: 1.0000e-04
Epoch 53/100
325/329 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0018
Epoch 53: val_loss did not improve from 0.00081
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0018 - val_loss: 8.1409e-04 - learning_rate: 1.0000e-04
Epoch 54/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0019
Epoch 54: val_loss improved from 0.00081 to 0.00079, saving model to /content/drive/MyDrive/ML/Signate_1634/lstm_model_no_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0019 - val_loss: 7.8840e-04 - learning_rate: 1.0000e-04
Epoch 55/100
327/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0019
Epoch 55: val_loss did not improve from 0.00079
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0019 - val_loss: 8.1010e-04 - learning_rate: 1.0000e-04
Epoch 56/100
325/329 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0019
Epoch 56: val_loss did not improve from 0.00079
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0019 - val_loss: 7.9255e-04 - learning_rate: 1.0000e-04
Epoch 57/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0018
Epoch 57: val_loss did not improve from 0.00079
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0018 - val_loss: 8.2673e-04 - learning_rate: 1.0000e-04
Epoch 58/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0018
Epoch 58: val_loss did not improve from 0.00079
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0018 - val_loss: 7.8960e-04 - learning_rate: 1.

### GRU モデル（PCA適用）

In [43]:
def build_gru_model(seq_length, n_features):
    model = Sequential([
        GRU(64, return_sequences=True, input_shape=(seq_length, n_features)),
        BatchNormalization(),
        Dropout(0.3),

        GRU(128, return_sequences=False),
        BatchNormalization(),
        Dropout(0.3),

        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# GRUモデルの構築と学習 (PCA適用データ)
gru_model_pca = build_gru_model(seq_length, X_train_seq_pca.shape[2])
print("\n--- GRUモデル (PCA適用) ---")
print(gru_model_pca.summary())

checkpoint_gru_pca = ModelCheckpoint(
    data_path + 'gru_model_pca.h5', # ファイル名を変更して区別
    monitor='val_loss',
    verbose=1,
    save_best_only=True
)

print("\n--- GRUモデル学習開始 (PCA適用) ---")
gru_history_pca = gru_model_pca.fit(
    X_train_seq_pca, y_train_seq_pca,
    epochs=100,
    batch_size=64,
    validation_data=(X_val_seq_pca, y_val_seq_pca),
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True),
        checkpoint_gru_pca,
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=1)
    ],
    verbose=1
)

# GRU 学習履歴のプロット部分 (PCA適用)
plt.figure(figsize=(7, 5))

plt.plot(gru_history_pca.history['loss'], label='Training Loss')
plt.plot(gru_history_pca.history['val_loss'], label='Validation Loss')
plt.title('GRU (PCA) - Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('gru_training_history_loss_pca.png') # ファイル名を変更
plt.close()


--- GRUモデル (PCA適用) ---


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_4 (GRU)                     │ (None, 24, 64)         │        15,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 24, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (None, 128)            │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 94,977 (371.00 KB)

 Trainable params: 94,593 (369.50 KB)

...

329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0067 - val_loss: 0.0013 - learning_rate: 0.0010
Epoch 14/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0065
Epoch 14: val_loss did not improve from 0.00131
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0065 - val_loss: 0.0021 - learning_rate: 0.0010
Epoch 15/100
325/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0060
Epoch 15: val_loss did not improve from 0.00131
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0060 - val_loss: 0.0018 - learning_rate: 0.0010
Epoch 16/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0055
Epoch 16: val_loss did not improve from 0.00131
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0055 - val_loss: 0.0018 - learning_rate: 0.0010
Epoch 17/100
323/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0051
Epoch 17: val_loss did not improve from 0.00131
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0051 - val_loss: 0.0022 - learning_rate: 0.0010
Epoch 18/100
326/329 ━━━━━━━━━━━━━━

329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0039 - val_loss: 0.0011 - learning_rate: 2.0000e-04
Epoch 20/100
323/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0038
Epoch 20: val_loss improved from 0.00114 to 0.00111, saving model to /content/drive/MyDrive/ML/Signate_1634/gru_model_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0038 - val_loss: 0.0011 - learning_rate: 2.0000e-04
Epoch 21/100
325/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0037
Epoch 21: val_loss did not improve from 0.00111
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0037 - val_loss: 0.0013 - learning_rate: 2.0000e-04
Epoch 22/100
327/329 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0036
Epoch 22: val_loss did not improve from 0.00111
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0036 - val_loss: 0.0011 - learning_rate: 2.0000e-04
Epoch 23/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0037
Epoch 23: val_loss did not improve from 0.00111
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0037 - val_loss: 0.0016 - learning_rate: 2.0000e-04
Epoch 24/100
327/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0037
Epoch 24: val_loss did not improve from 0.00111

Epoch 24: ReduceLROnPlateau reducing learning rate to 0.0001.
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.00

329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0034 - val_loss: 9.9292e-04 - learning_rate: 1.0000e-04
Epoch 26/100
323/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0034
Epoch 26: val_loss did not improve from 0.00099
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0034 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 27/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0033
Epoch 27: val_loss did not improve from 0.00099
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0033 - val_loss: 0.0011 - learning_rate: 1.0000e-04
Epoch 28/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0032
Epoch 28: val_loss improved from 0.00099 to 0.00099, saving model to /content/drive/MyDrive/ML/Signate_1634/gru_model_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0032 - val_loss: 9.9079e-04 - learning_rate: 1.0000e-04
Epoch 29/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0033
Epoch 29: val_loss did not improve from 0.00099
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0033 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 30/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0032
Epoch 30: val_loss did not improve from 0.00099
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0032 - val_loss: 0.0011 - learning_rate: 1.0000e-04
Epoch 31/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0033
Epoch 31: val_loss did not improve from 0.00099
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0033 - val_loss: 0.0011 - learning_rate: 1.0000e-04
Epoch 32/100
323/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0033
Epoch 32: val_loss did not improve from 0.00099
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0033 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 33/1

### GRU モデル（PCA適用なし

In [44]:
# GRUモデルの構築と学習 (PCA適用なしデータ)
gru_model_no_pca = build_gru_model(seq_length, X_train_seq_no_pca.shape[2])
print("\n--- GRUモデル (PCA適用なし) ---")
print(gru_model_no_pca.summary())

checkpoint_gru_no_pca = ModelCheckpoint(
    data_path + 'gru_model_no_pca.h5', # ファイル名を変更して区別
    monitor='val_loss',
    verbose=1,
    save_best_only=True
)


print("\n--- GRUモデル学習開始 (PCA適用なし) ---")
gru_history_no_pca = gru_model_no_pca.fit(
    X_train_seq_no_pca, y_train_seq_no_pca,
    epochs=100,
    batch_size=64,
    validation_data=(X_val_seq_no_pca, y_val_seq_no_pca),
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True),
        checkpoint_gru_no_pca,
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=1)
    ],
    verbose=1
)

# GRU 学習履歴のプロット部分 (PCA適用なし)
plt.figure(figsize=(7, 5))

plt.plot(gru_history_no_pca.history['loss'], label='Training Loss')
plt.plot(gru_history_no_pca.history['val_loss'], label='Validation Loss')
plt.title('GRU (No PCA) - Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('gru_training_history_loss_no_pca.png') # ファイル名を変更
plt.close()



--- GRUモデル (PCA適用なし) ---


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_6 (GRU)                     │ (None, 24, 64)         │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 24, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_7 (GRU)                     │ (None, 128)            │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97,857 (382.25 KB)

 Trainable params: 97,473 (380.75 KB)

...

329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0022 - val_loss: 6.8946e-04 - learning_rate: 2.0000e-04
Epoch 26/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0021
Epoch 26: val_loss improved from 0.00069 to 0.00066, saving model to /content/drive/MyDrive/ML/Signate_1634/gru_model_no_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0021 - val_loss: 6.5673e-04 - learning_rate: 2.0000e-04
Epoch 27/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0020
Epoch 27: val_loss improved from 0.00066 to 0.00063, saving model to /content/drive/MyDrive/ML/Signate_1634/gru_model_no_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0020 - val_loss: 6.3001e-04 - learning_rate: 2.0000e-04
Epoch 28/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0021
Epoch 28: val_loss improved from 0.00063 to 0.00063, saving model to /content/drive/MyDrive/ML/Signate_1634/gru_model_no_pca.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0021 - val_loss: 6.2842e-04 - learning_rate: 2.0000e-04
Epoch 29/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0020
Epoch 29: val_loss did not improve from 0.00063
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0020 - val_loss: 6.4790e-04 - learning_rate: 2.0000e-04
Epoch 30/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0020
Epoch 30: val_loss did not improve from 0.00063

Epoch 30: ReduceLROnPlateau reducing learning rate to 0.0001.
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0020 - val_loss: 6.3078e-04 - learning_rate: 2.0000e-04
Epoch 31/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0019
Epoch 31: val_loss did not improve from 0.00063
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0019 - val_loss: 6.4105e-04 - learning_rate: 1.0000e-04
Epoch 32/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0019
Epoch 32: val_loss did not improve from 0.00063
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/ste

329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0019 - val_loss: 6.1004e-04 - learning_rate: 1.0000e-04
Epoch 34/100
327/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0018
Epoch 34: val_loss did not improve from 0.00061
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0018 - val_loss: 6.7124e-04 - learning_rate: 1.0000e-04
Epoch 35/100
325/329 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0018
Epoch 35: val_loss did not improve from 0.00061
329/329 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 0.0018 - val_loss: 6.7897e-04 - learning_rate: 1.0000e-04
Epoch 36/100
325/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0018
Epoch 36: val_loss did not improve from 0.00061
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0018 - val_loss: 6.6013e-04 - learning_rate: 1.0000e-04
Epoch 37/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0018
Epoch 37: val_loss did not improve from 0.00061
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0018 - val_loss: 6.6638e-04 - learning_rate: 1.000

## モデル評価と結果比較

In [45]:
def evaluate_model(model, X_test_seq, y_test_seq_scaled, y_scaler, title): # y_test_scaledをy_test_seq_scaledに変更
    """
    モデルを評価し、MAEとRMSEを計算、結果をプロットする
    """
    # 予測
    scaled_predictions = model.predict(X_test_seq)

    # スケーリングを元に戻す
    # MAE/RMSE計算とプロットのために、y_test_seq_scaledを使用する
    y_test_orig = y_scaler.inverse_transform(y_test_seq_scaled.reshape(-1, 1)).flatten() # y_test_scaledをy_test_seq_scaledに変更
    y_pred_orig = y_scaler.inverse_transform(scaled_predictions).flatten()

    # 評価指標
    mae = mean_absolute_error(y_test_orig, y_pred_orig)
    rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))

    print(f"\n--- {title} 評価結果 ---")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")

    # 予測結果と実測値の比較プロット
    plt.figure(figsize=(15, 6))
    # プロットの実測値もy_test_seq_scaledから計算する
    plt.plot(y_scaler.inverse_transform(y_test_seq_scaled.reshape(-1, 1)).flatten(), label='Actual Price') # y_test_scaledをy_test_seq_scaledに変更
    plt.plot(y_pred_orig, label='Predicted Price')
    plt.title(f'{title} - 実測値 vs 予測値')
    plt.xlabel('Time Step')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{title.replace(" ", "_")}_prediction_vs_actual.png')
    plt.close()

    return mae, rmse

# LSTM (PCA適用) の評価
# y_test_scaled_pca を y_test_seq_pca に変更して evaluate_model を呼び出す
lstm_pca_mae, lstm_pca_rmse = evaluate_model(lstm_model_pca, X_test_seq_pca, y_test_seq_pca, y_scaler, "LSTM (PCA)")

# LSTM (PCA適用なし) の評価
# y_test_scaled_no_pca を y_test_seq_no_pca に変更して evaluate_model を呼び出す
lstm_no_pca_mae, lstm_no_pca_rmse = evaluate_model(lstm_model_no_pca, X_test_seq_no_pca, y_test_seq_no_pca, y_scaler, "LSTM (No PCA)")

# GRU (PCA適用) の評価
# y_test_scaled_pca を y_test_seq_pca に変更して evaluate_model を呼び出す
gru_pca_mae, gru_pca_rmse = evaluate_model(gru_model_pca, X_test_seq_pca, y_test_seq_pca, y_scaler, "GRU (PCA)")

# GRU (PCA適用なし) の評価
# y_test_scaled_no_pca を y_test_seq_no_pca に変更して evaluate_model を呼び出す
gru_no_pca_mae, gru_no_pca_rmse = evaluate_model(gru_model_no_pca, X_test_seq_no_pca, y_test_seq_no_pca, y_scaler, "GRU (No PCA)")

# 結果の比較
print("\n--- モデル比較結果 ---")
print(f"LSTM (PCA):     MAE={lstm_pca_mae:.4f}, RMSE={lstm_pca_rmse:.4f}")
print(f"LSTM (No PCA):  MAE={lstm_no_pca_mae:.4f}, RMSE={lstm_no_pca_rmse:.4f}")
print(f"GRU (PCA):      MAE={gru_pca_mae:.4f}, RMSE={gru_pca_rmse:.4f}")
print(f"GRU (No PCA):   MAE={gru_no_pca_mae:.4f}, RMSE={gru_no_pca_rmse:.4f}")

82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

--- LSTM (PCA) 評価結果 ---
MAE: 6.2338
RMSE: 8.1579
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

--- LSTM (No PCA) 評価結果 ---
MAE: 3.6980
RMSE: 5.2392
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

--- GRU (PCA) 評価結果 ---
MAE: 4.0530
RMSE: 5.5660
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

--- GRU (No PCA) 評価結果 ---
MAE: 3.0711
RMSE: 4.3489

--- モデル比較結果 ---
LSTM (PCA):     MAE=6.2338, RMSE=8.1579
LSTM (No PCA):  MAE=3.6980, RMSE=5.2392
GRU (PCA):      MAE=4.0530, RMSE=5.5660
GRU (No PCA):   MAE=3.0711, RMSE=4.3489


## 特徴量重要度の分析

In [46]:
# 特徴量重要度の可視化（SelectKBestの結果）
def plot_feature_importance(X, y, top_n=20):
    # 数値型以外のカラムを削除
    X_numeric = X.select_dtypes(include=np.number)

    selector = SelectKBest(score_func=f_regression, k='all')
    # 数値型カラムのみを使ってfitを行う
    selector.fit(X_numeric, y)

    # 特徴量のスコアを取得
    feature_scores = pd.DataFrame({
        'Feature': X_numeric.columns, # X_numericのカラム名を使用
        'Score': selector.scores_
    })

    # スコア順にソート
    feature_scores = feature_scores.sort_values(by='Score', ascending=False)

    # 上位n個の特徴量をプロット
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Score', y='Feature', data=feature_scores.head(top_n))
    plt.title(f'上位{top_n}個の重要な特徴量')
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    plt.close()

    return feature_scores

# 特徴量重要度のプロット
# XがDataFrameであり、数値型以外のカラムを含んでいる可能性があるため、関数内で処理します
importance = plot_feature_importance(X_featured, y_featured, top_n=20)
print("\n上位20の重要な特徴量:")
print(importance.head(20))


上位20の重要な特徴量:
                                        Feature          Score
84                           price_actual_lag_1  362397.475584
85                          price_actual_lag_24   55455.839343
88                 price_actual_rolling_mean_24   51125.358795
87                         price_actual_lag_168   46096.371945
86                          price_actual_lag_48   29553.563560
3                   generation_fossil_hard_coal    7410.283193
1          generation_fossil_brown_coal/lignite    4315.279936
14                            total_load_actual    4191.590208
2                         generation_fossil_gas    4151.326152
5   generation_hydro_pumped_storage_consumption    2642.416186
6    generation_hydro_run_of_river_and_poundage    1905.413869
79                                      quarter    1801.790531
4                         generation_fossil_oil    1735.021199
75                                         hour    1639.636595
81                                   week

## 時系列交差検証

In [47]:
# 時系列交差検証
def time_series_cv_evaluation(X, y, seq_length=24, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)

    mae_scores = []
    rmse_scores = []
    fold = 1

    for train_index, test_index in tscv.split(X):
        print(f"\n評価 fold {fold}/{n_splits}")

        # データの分割
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]

        # スケーリング
        y_scaler_fold = MinMaxScaler()
        y_train_scaled = y_scaler_fold.fit_transform(y_train_fold.reshape(-1, 1))
        y_test_scaled = y_scaler_fold.transform(y_test_fold.reshape(-1, 1))

        # シーケンスデータの作成
        X_train_seq_fold, y_train_seq_fold = create_sequences(X_train_fold, y_train_scaled, seq_length)
        X_test_seq_fold, y_test_seq_fold = create_sequences(X_test_fold, y_test_scaled, seq_length)

        if len(X_train_seq_fold) == 0 or len(X_test_seq_fold) == 0:
            print("シーケンスデータが空です")
            continue

        # モデル構築と学習
        model = build_lstm_model(seq_length, X_train_seq_fold.shape[2])

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, restore_best_weights=True)

        model.fit(
            X_train_seq_fold, y_train_seq_fold,
            epochs=50,
            batch_size=64,
            validation_split=0.2,
            callbacks=[early_stopping],
            verbose=0
        )

        # 評価
        y_pred_fold = model.predict(X_test_seq_fold)

        # スケーリングを元に戻す
        y_test_orig_fold = y_scaler_fold.inverse_transform(y_test_seq_fold.reshape(-1, 1)).flatten()
        y_pred_orig_fold = y_scaler_fold.inverse_transform(y_pred_fold).flatten()

        # 評価指標の計算
        mae = mean_absolute_error(y_test_orig_fold, y_pred_orig_fold)
        rmse = np.sqrt(mean_squared_error(y_test_orig_fold, y_pred_orig_fold))

        mae_scores.append(mae)
        rmse_scores.append(rmse)

        print(f"Fold {fold} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
        fold += 1

    # 結果をまとめる
    print("\n時系列交差検証の結果:")
    print(f"平均 MAE: {np.mean(mae_scores):.2f} (±{np.std(mae_scores):.2f})")
    print(f"平均 RMSE: {np.mean(rmse_scores):.2f} (±{np.std(rmse_scores):.2f})")

    return mae_scores, rmse_scores

# 時系列交差検証の実行
mae_cv, rmse_cv = time_series_cv_evaluation(X_pca, y_scaled, seq_length=24, n_splits=3)


評価 fold 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Fold 1 - MAE: 0.08, RMSE: 0.11

評価 fold 2/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Fold 2 - MAE: 0.05, RMSE: 0.07

評価 fold 3/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Fold 3 - MAE: 0.04, RMSE: 0.05

時系列交差検証の結果:
平均 MAE: 0.06 (±0.02)
平均 RMSE: 0.07 (±0.02)


### 追加特徴量の作成

In [48]:
def create_additional_features_combined(df, is_test=False, train_df=None):
    """
    時系列データに対して時間系特徴量・ラグ特徴量・統計特徴量を追加

    Parameters:
    - df: 特徴量追加対象のDataFrame（DatetimeIndexを持つ）
    - is_test: testデータでの呼び出しかどうか
    - train_df: trainデータ（is_test=True のとき、ラグ計算用に参照）

    Returns:
    - df_features: 特徴量が追加されたDataFrame
    """
    df_features = df.copy()

    # インデックスの整備（UTC前提）
    if not isinstance(df_features.index, pd.DatetimeIndex) or df_features.index.tz is not None:
        df_features.index = pd.to_datetime(df_features.index, utc=True)

    # 基本的な時間特徴量
    df_features['hour'] = df_features.index.hour
    df_features['dayofweek'] = df_features.index.dayofweek
    df_features['month'] = df_features.index.month
    df_features['year'] = df_features.index.year
    df_features['quarter'] = df_features.index.quarter
    df_features['dayofyear'] = df_features.index.dayofyear
    df_features['weekofyear'] = df_features.index.isocalendar().week.astype(int)
    df_features['dayofmonth'] = df_features.index.day
    df_features['is_weekend'] = (df_features.index.dayofweek >= 5).astype(int)

    # ラグ特徴量
    if is_test and train_df is not None:
        # train_dfの末尾と結合してラグを引き継ぐ
        df_lag = pd.concat([train_df[['price_actual']].copy(), df_features], axis=0)
    else:
        df_lag = df_features.copy()

    df_lag['price_actual_lag_1'] = df_lag['price_actual'].shift(1)
    df_lag['price_actual_lag_24'] = df_lag['price_actual'].shift(24)
    df_lag['price_actual_lag_48'] = df_lag['price_actual'].shift(48)
    df_lag['price_actual_lag_168'] = df_lag['price_actual'].shift(168)

    # 移動平均
    df_lag['price_actual_rolling_mean_24'] = df_lag['price_actual'].rolling(window=24).mean()

    # testの場合、trainを除いた test 部分だけを返す
    if is_test and train_df is not None:
        df_result = df_lag.loc[df_features.index]
    else:
        df_result = df_lag

    # 欠損値処理（主にラグ・rolling平均による）
    df_result = df_result.ffill().bfill()

    return df_result


In [49]:
from tqdm import tqdm

def recursive_forecast(model, test_df, train_df, pipeline):
    """
    学習データの末尾を使って test を自己回帰的に予測
    各ステップで特徴量生成 + 前処理 + 予測 + price_actual の更新 を行う
    """
    seq_length = pipeline['seq_length']
    selected_features = pipeline['selected_features']
    scaler = pipeline['pca_scaler']
    y_scaler = pipeline['y_scaler']
    pca = pipeline['pca']

    df_past = train_df.copy()
    df_future = test_df.copy()

    preds = []

    for i in tqdm(range(len(df_future))):
        # 今の時点の未来データ1ステップ
        step_df = df_future.iloc[[i]].copy()

        # 特徴量生成
        df_with_feature = create_additional_features_combined(
            step_df, is_test=True, train_df=df_past
        )

        # 特徴量選択
        X_step = df_with_feature[selected_features]

        # スケーリング
        X_scaled = scaler.transform(X_step)

        # PCA適用
        if pca is not None:
            X_scaled = pca.transform(X_scaled)

        # シーケンス作成
        if len(df_past) >= seq_length - 1:
            recent = df_past.tail(seq_length - 1)[['price_actual']].copy()
            recent_features = []

            for j in range(seq_length - 1):
                step = df_past.iloc[-(seq_length - 1 - j)]
                step_feat = create_additional_features_combined(
                    step.to_frame().T, is_test=True, train_df=df_past.iloc[:-(seq_length - 1 - j)]
                )
                feat = step_feat[selected_features]
                feat_scaled = scaler.transform(feat)
                feat_scaled = pca.transform(feat_scaled) if pca is not None else feat_scaled
                recent_features.append(feat_scaled[0])

            recent_features.append(X_scaled[0])
            X_seq = np.array(recent_features).reshape(1, seq_length, -1)

            # 予測
            y_pred_scaled = model.predict(X_seq)
            y_pred = y_scaler.inverse_transform(y_pred_scaled)[0, 0]
        else:
            y_pred = df_past['price_actual'].iloc[-1]  # fallback

        preds.append(y_pred)

        # 予測結果を df_future に反映
        df_future.at[step_df.index[0], 'price_actual'] = y_pred
        df_past = pd.concat([df_past, df_future.iloc[[i]]])

    return preds


## モデルとパイプラインの保存

In [50]:
import pickle

# モデル保存のためのパイプラインを作成
pipeline = {
    'pca': pca,
    'pca_scaler': feature_scaler ,
    'y_scaler': y_scaler,
    'selected_features': X_selected_scaled.columns.tolist(),
    'seq_length': seq_length
}

# パイプラインを保存
pipeline_save_path = data_path + 'price_prediction_pipeline.pkl'
with open(pipeline_save_path, 'wb') as f:
    pickle.dump(pipeline, f)

# 予測パイプラインの使用例
def predict_price(model, new_data, pipeline):
    """
    新しいデータで価格を予測する

    Parameters:
    - model: 訓練済みモデル
    - new_data: 予測したいデータ（必要な特徴量を含むDataFrame）
    - pipeline: 前処理パイプライン

    Returns:
    - predicted_price: 予測された価格
    """
    # パイプラインから必要な要素を取得
    pca = pipeline['pca']
    pca_scaler = pipeline['pca_scaler']
    y_scaler = pipeline['y_scaler']
    selected_features = pipeline['selected_features']
    seq_length = pipeline['seq_length']

    # 必要な特徴量のみを選択
    selected_data = new_data[selected_features]

    # スケーリングとPCA変換
    scaled_data = pca_scaler.transform(selected_data)
    pca_data = pca.transform(scaled_data)

    # シーケンスデータの作成（最新のseq_length分のデータを使用）
    if len(pca_data) >= seq_length:
        sequence = pca_data[-seq_length:].reshape(1, seq_length, -1)

        # 予測
        scaled_prediction = model.predict(sequence)

        # スケーリングを元に戻す
        prediction = y_scaler.inverse_transform(scaled_prediction)[0, 0]

        return prediction
    else:
        raise ValueError(f"入力データが短すぎます。少なくとも{seq_length}点必要です。")
# 特徴量選択名を取得（再取得）
selected_features_for_inference = feature_selector.get_support(indices=True)
selected_feature_names = X_featured.select_dtypes(include=np.number).columns[selected_features_for_inference].tolist()

print("\nモデルと前処理パイプラインを保存しました。")
print("新しいデータに対する予測は 'predict_price' 関数を使用して行えます。")


モデルと前処理パイプラインを保存しました。
新しいデータに対する予測は 'predict_price' 関数を使用して行えます。


### 最適モデルの決定と最終予測

In [51]:
#------------------------------------------------------------
# ★ 評価結果の取りまとめ 〜 最適モデル決定
#------------------------------------------------------------
evaluation_results = {
    "LSTM (PCA)": {"mae": lstm_pca_mae, "rmse": lstm_pca_rmse,
                   "model": lstm_model_pca, "data_type": "pca"},
    "LSTM (No PCA)": {"mae": lstm_no_pca_mae, "rmse": lstm_no_pca_rmse,
                      "model": lstm_model_no_pca, "data_type": "no_pca"},
    "GRU  (PCA)": {"mae": gru_pca_mae, "rmse": gru_pca_rmse,
                   "model": gru_model_pca, "data_type": "pca"},
    "GRU  (No PCA)": {"mae": gru_no_pca_mae, "rmse": gru_no_pca_rmse,
                      "model": gru_model_no_pca, "data_type": "no_pca"}
}

print("\n--- モデル比較結果 ---")
for k, v in evaluation_results.items():
    print(f"{k}: MAE={v['mae']:.4f}, RMSE={v['rmse']:.4f}")

best_model_name = min(evaluation_results, key=lambda k: evaluation_results[k]['mae'])
best_info       = evaluation_results[best_model_name]
best_model      = best_info['model']
best_data_type  = best_info['data_type']
loaded_seq_len  = seq_length            # そのまま保持

print(f"\n最適なモデルは: {best_model_name} (MAE={best_info['mae']:.4f})")

#------------------------------------------------------------
# ★ スケーラー & PCA
#------------------------------------------------------------
final_pca            = pca if best_data_type == "pca" else None
final_feature_scaler = feature_scaler
final_y_scaler       = y_scaler
print("最終予測にPCAを使用します。" if final_pca else "最終予測にPCAは使用しません。")

#------------------------------------------------------------
# ★ テストデータ：特徴量生成
#------------------------------------------------------------
test_df_spain = (
    test_df[test_df["item_ID"] == "spain_total"]
    .ffill().bfill()
    .drop(columns="item_ID")
)

test_feat = create_additional_features_combined(
    test_df_spain.copy(), is_test=True, train_df=df.copy()
).fillna(0)
test_feat["price_actual"] = np.nan   # 予測を書き込むダミー列

#------------------------------------------------------------
# ★ train データも同じ関数で “フル特徴量” を作る
#------------------------------------------------------------
train_feat = create_additional_features_combined(df.copy(), is_test=False).fillna(0)

#------------------------------------------------------------
# ★ 選択された特徴量リスト
#------------------------------------------------------------
selected_cols = pipeline["selected_features"]

missing_test  = set(selected_cols) - set(test_feat.columns)
missing_train = set(selected_cols) - set(train_feat.columns)
if missing_test or missing_train:
    raise ValueError(f"列が足りません\n test → {missing_test}\n train → {missing_train}")

#------------------------------------------------------------
# ★ ① 追加で保存するリスト
#------------------------------------------------------------
all_numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
selected_cols    = pipeline["selected_features"]               # 30 列
sel_idx          = [all_numeric_cols.index(c) for c in selected_cols]

# pipeline を上書き保存（あとで読み直す場合に備えて）
pipeline.update({
    "all_numeric_cols": all_numeric_cols,
    "selected_idx"    : sel_idx
})

#------------------------------------------------------------
# ★ ② 逐次予測関数：139列でスケール → 30列だけ抽出
#------------------------------------------------------------
def recursive_forecast(model, test_df_full, train_df_full, pipe):

    pca        = pipe["pca"]
    scaler     = pipe["pca_scaler"]
    y_scaler   = pipe["y_scaler"]
    feats      = pipe["selected_features"]      # 30 列名
    all_cols   = pipe["all_numeric_cols"]       # 139 列名
    sel_idx    = pipe["selected_idx"]           # 30 → 139 中の位置
    seq_len    = pipe["seq_length"]

    # index を tz なしに統一
    train_df_full.index = pd.to_datetime(train_df_full.index).tz_localize(None)
    test_df_full.index  = pd.to_datetime(test_df_full.index).tz_localize(None)

    full  = train_df_full.copy()
    preds = []

    for ts, row in test_df_full.iterrows():

        # ❶ 末尾に 1 行追加（列合わせ）
        full = pd.concat(
            [full, row.to_frame().T.reindex(columns=full.columns)],
            copy=False
        )

        # ❷ 最新行 & 履歴を 139 列でそろえて取り出す ----------------
        x_num   = full.loc[ts].reindex(all_cols).fillna(0)
        hist_num= full[all_cols].iloc[-(seq_len-1):].fillna(0)

        # ❸ スケール（139→139）------------------------------------
        x_scaled_all    = scaler.transform(x_num.to_frame().T)      # (1,139)
        hist_scaled_all = scaler.transform(hist_num)               # (seq-1,139)

        # ❹ 30 列だけ切り出し -------------------------------------
        x_scaled_sel    = x_scaled_all[:, sel_idx]                  # (1,30)
        hist_scaled_sel = hist_scaled_all[:, sel_idx]               # (seq-1,30)

        # ❺ PCA（必要なら）-----------------------------------------
        if pca:
            x_final    = pca.transform(x_scaled_sel)
            hist_final = pca.transform(hist_scaled_sel)
        else:
            x_final, hist_final = x_scaled_sel, hist_scaled_sel

        seq = np.vstack([hist_final, x_final]).reshape(1, seq_len, -1)

        # ❻ 予測 ---------------------------------------------------
        y_scaled = model.predict(seq, verbose=0)
        y_pred   = y_scaler.inverse_transform(y_scaled)[0, 0]
        preds.append(y_pred)

        # ❼ 予測値 & ラグ -----------------------------------------
        full.iat[-1, full.columns.get_loc("price_actual")] = y_pred
        idx_cur   = len(full) - 1
        col_price = full.columns.get_loc("price_actual")
        for lag in (1, 24, 48, 168):
            lag_col = f"price_actual_lag_{lag}"
            if lag_col in full.columns and idx_cur >= lag:
                col_pos = full.columns.get_loc(lag_col)
                full.iat[idx_cur, col_pos] = full.iat[idx_cur - lag, col_price]

    return preds



#------------------------------------------------------------
# ★ 推論実行
#------------------------------------------------------------
preds = recursive_forecast(
    best_model,
    test_feat.copy(),
    train_feat.copy(),
    {
        "pca"              : final_pca,
        "pca_scaler"       : final_feature_scaler,
        "y_scaler"         : final_y_scaler,
        "selected_features": selected_cols,
        "seq_length"       : loaded_seq_len,
    }
)

#------------------------------------------------------------
# ★ 提出ファイル作成
#------------------------------------------------------------
sub_df = pd.DataFrame({
    "id"          : [ts.strftime("%Y-%m-%d %H:%M:%S") + "_spain_total"
                     for ts in test_df_spain.index],
    "price_actual": preds
})
sub_path = data_path + "submission.csv"
sub_df.to_csv(sub_path, header=False, index=False)

print(f"\n最終推論結果を '{sub_path}' に保存しました。")
display(sub_df.head())



--- モデル比較結果 ---
LSTM (PCA): MAE=6.2338, RMSE=8.1579
LSTM (No PCA): MAE=3.6980, RMSE=5.2392
GRU  (PCA): MAE=4.0530, RMSE=5.5660
GRU  (No PCA): MAE=3.0711, RMSE=4.3489

最適なモデルは: GRU  (No PCA) (MAE=3.0711)
最終予測にPCAは使用しません。


ValueError: 'hour' is not in list

## 最終提出データの作成

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
import tensorflow as tf

# test_dfに対する推論
# test_IDが'spain_total'のデータのみを選択
test_df_spain = test_df[test_df['item_ID'] == 'spain_total'].copy()

# 欠損値処理 (trainと同じ処理を適用)
test_df_spain = test_df_spain.ffill()
test_df_spain = test_df_spain.bfill()

# 'item_ID' 列を削除
test_df_spain = test_df_spain.drop('item_ID', axis=1)

# 保存したパイプラインを読み込み
pipeline_load_path = data_path + 'price_prediction_pipeline.pkl'
loaded_pipeline = None
loaded_selected_features = None

if os.path.exists(pipeline_load_path):
    try:
        with open(pipeline_load_path, 'rb') as f:
            loaded_pipeline = pickle.load(f)

        loaded_pca = loaded_pipeline['pca']
        loaded_pca_scaler = loaded_pipeline['pca_scaler']
        loaded_y_scaler = loaded_pipeline['y_scaler']
        loaded_selected_features = loaded_pipeline['selected_features']
        loaded_seq_length = loaded_pipeline['seq_length']

        print(f"\nパイプラインをロードしました。学習時に選択された特徴量 ({len(loaded_selected_features)}個): {loaded_selected_features}")

    except Exception as e:
        print(f"パイプラインのロードに失敗しました: {e}")
        loaded_pipeline = None
else:
    print(f"エラー: パイプラインファイル '{pipeline_load_path}' が見つかりません。")

if loaded_pipeline is not None and loaded_selected_features is not None:
    # 学習データの特徴量エンジニアリング済みデータが必要
    # dfに対して同じ特徴量エンジニアリングを適用
    if 'df' in locals() or 'df' in globals():
        # 学習データに特徴量エンジニアリングを適用（price_actualラグを含む）
        df_featured_train = create_additional_features_combined(df.copy(), is_test=False)

        # テストデータに特徴量エンジニアリングを適用（学習データを使ってラグを計算）
        test_df_featured = create_additional_features_combined(
            test_df_spain.copy(),
            is_test=True,
            train_df=df.copy()
        )

        print(f"テストデータの特徴量エンジニアリング完了。形状: {test_df_featured.shape}")

        # NaN値の確認
        nan_count = test_df_featured.isnull().sum().sum()
        print(f"テストデータのNaN値の総数: {nan_count}")

        if nan_count > 0:
            print("NaN値が残っているカラム:")
            nan_columns = test_df_featured.isnull().sum()
            print(nan_columns[nan_columns > 0])

            # 残りのNaN値を0で埋める
            test_df_featured = test_df_featured.fillna(0)
            print("残りのNaN値を0で埋めました。")

        try:
            # 学習時に選択された特徴量のみを抽出
            test_data_aligned = test_df_featured[loaded_selected_features]
            print(f"テストデータ ({test_data_aligned.shape[1]}個の特徴量) のカラム順序を合わせました。")

            # 最終的なNaN確認
            final_nan_count = test_data_aligned.isnull().sum().sum()
            print(f"最終的なNaN値の総数: {final_nan_count}")

            if final_nan_count > 0:
                print("警告: まだNaN値が残っています。さらに処理します。")
                test_data_aligned = test_data_aligned.fillna(method='ffill').fillna(method='bfill').fillna(0)

        except KeyError as e:
            print(f"エラー: 学習時に選択された特徴量 '{e}' がテストデータに見つかりません。")
            test_data_aligned = None

        if test_data_aligned is not None:
            # モデルをロード
            lstm_model_path = data_path + 'lstm_model.h5'
            try:
                loaded_model = tf.keras.models.load_model(
                    lstm_model_path,
                    custom_objects={'mse': tf.keras.losses.MeanSquaredError()}
                )
                print("モデルを正常にロードしました。")

                # テストデータに前処理を適用
                test_data_scaled = loaded_pca_scaler.transform(test_data_aligned)
                test_data_pca = loaded_pca.transform(test_data_scaled)
                print(f"テストデータにスケーリングとPCAを適用しました。形状: {test_data_pca.shape}")

                # 学習データのPCA結果と結合
                if 'X_pca' in locals() or 'X_pca' in globals():
                    combined_data_pca = np.concatenate([X_pca, test_data_pca])
                    print(f"データを結合しました。形状: {combined_data_pca.shape}")

                    # 推論用シーケンスの作成
                    X_inference = []
                    start_index_test = len(X_pca)

                    for i in range(len(test_data_pca)):
                        seq_start = start_index_test + i - loaded_seq_length
                        seq_end = start_index_test + i

                        if seq_start >= 0:
                            input_sequence = combined_data_pca[seq_start:seq_end]
                            X_inference.append(input_sequence)

                    if X_inference:
                        X_inference = np.array(X_inference)
                        print(f"推論用シーケンスデータ形状: {X_inference.shape}")

                        # 予測実行
                        scaled_predictions = loaded_model.predict(X_inference)
                        predictions = loaded_y_scaler.inverse_transform(scaled_predictions).flatten()

                        # 提出ファイル作成
                        valid_test_indices = test_df_spain.index[len(test_df_spain) - len(predictions):]
                        submission_df = pd.DataFrame({
                            'id': [ts.strftime('%Y-%m-%d %H:%M:%S') for ts in valid_test_indices],
                            'price_actual': predictions
                        })

                        submission_path = data_path + 'submission.csv'
                        submission_df.to_csv(submission_path, header=False, index=False)

                        print(f"\n推論結果を '{submission_path}' に保存しました。")
                        print("提出ファイルの先頭:")
                        display(pd.read_csv(submission_path, header=None).head())

                    else:
                        print("エラー: 予測のための入力シーケンスが作成できませんでした。")
                else:
                    print("エラー: 学習データのPCA結果 (X_pca) が見つかりません。")

            except Exception as e:
                print(f"モデルのロードまたは推論に失敗しました: {e}")
    else:
        print("エラー: 学習データ (df) が見つかりません。")
else:
    print("パイプラインがロードされなかったため、推論を実行できません。")

### 最終メモ
この修正により、PCAによる次元削減がモデル性能にどのような影響を与えるかを定量的に比較できるようになります。

次に進めるべきステップとしては、ハイパーパラメータ検索の導入や、時系列交差検証のロジック一本化・ログ強化が考えられます。どちらに進みますか？ または、今回のコードについてさらに詳細な説明や修正が必要な点があればお知らせください。

In [ ]:
# 1. 推論対象の特徴量と学習時特徴量の一致をチェック
print(test_data_aligned.columns.tolist() == loaded_selected_features)

# 2. テストデータの最初のラグ特徴量確認
print(test_df_featured[['price_actual_lag_1', 'price_actual_lag_24']].head())

# 3. シーケンスの中身確認（shapeと内容）
print(X_inference.shape)
print(X_inference[0][-1])  # 最後のタイムステップの値

# 4. モデルの確認
print(best_model_name)  # 'GRU (PCA)' になってるか
